In [ ]:
# ============================================================================
# AUTOMATIZACIÓN CON GOOGLE SHEETS Y EMAIL
# ============================================================================

def crear_banner(titulo, icono="✨"):
    """Crea un banner simple para secciones"""
    print("\n" + "=" * 80)
    print(f" {icono} {titulo} {icono}")
    print("=" * 80)

crear_banner("GOOGLE SHEETS + EMAIL: AUTOMATIZACIÓN PODEROSA", "📊")

print(" **¿Por qué Google Sheets + Email?**")
print("   • Google Sheets: Base de datos accesible y colaborativa")
print("   • Email: Canal de comunicación universal")
print("   • Combinación: Reportes automáticos y actualizaciones en tiempo real")
print()

# ============================================================================
# PARTE 1: FUNDAMENTOS DE GOOGLE SHEETS PARA AUTOMATIZACIÓN
# ============================================================================

print(" **GOOGLE SHEETS COMO CENTRO DE DATOS**")
print("=" * 50)

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import io
import base64

# Simulador de Google Sheets API
class GoogleSheetsSimulator:
    """Simulador de Google Sheets para fines educativos"""

    def __init__(self):
        self.hojas = {}
        self.triggers_configurados = []

    def crear_hoja(self, nombre, datos_iniciales=None):
        """Crear una nueva hoja de cálculo"""
        if datos_iniciales is None:
            datos_iniciales = pd.DataFrame()

        self.hojas[nombre] = {
            'datos': datos_iniciales.copy(),
            'metadatos': {
                'creada': datetime.now(),
                'ultima_modificacion': datetime.now(),
                'filas': len(datos_iniciales),
                'columnas': len(datos_iniciales.columns) if not datos_iniciales.empty else 0
            }
        }

        print(f" Hoja '{nombre}' creada exitosamente")
        print(f"    Dimensiones: {len(datos_iniciales)} filas x {len(datos_iniciales.columns) if not datos_iniciales.empty else 0} columnas")

    def agregar_fila(self, nombre_hoja, nueva_fila):
        """Agregar una nueva fila a la hoja"""
        if nombre_hoja not in self.hojas:
            print(f" Hoja '{nombre_hoja}' no existe")
            return False

        hoja = self.hojas[nombre_hoja]

        # Convertir nueva_fila a DataFrame si es necesario
        if isinstance(nueva_fila, dict):
            nueva_fila_df = pd.DataFrame([nueva_fila])
        elif isinstance(nueva_fila, list):
            # Asumir que coincide con las columnas existentes
            if not hoja['datos'].empty:
                nueva_fila_df = pd.DataFrame([nueva_fila], columns=hoja['datos'].columns)
            else:
                nueva_fila_df = pd.DataFrame([nueva_fila])
        else:
            nueva_fila_df = nueva_fila

        # Agregar la fila
        hoja['datos'] = pd.concat([hoja['datos'], nueva_fila_df], ignore_index=True)
        hoja['metadatos']['ultima_modificacion'] = datetime.now()
        hoja['metadatos']['filas'] = len(hoja['datos'])

        print(f" Fila agregada a '{nombre_hoja}'")
        print(f"    Nueva fila: {nueva_fila}")

        # Verificar triggers
        self.verificar_triggers(nombre_hoja, 'nueva_fila', nueva_fila)

        return True

    def actualizar_fila(self, nombre_hoja, indice, nuevos_datos):
        """Actualizar una fila existente"""
        if nombre_hoja not in self.hojas:
            print(f" Hoja '{nombre_hoja}' no existe")
            return False

        hoja = self.hojas[nombre_hoja]

        if indice >= len(hoja['datos']):
            print(f" Índice {indice} fuera de rango")
            return False

        # Actualizar la fila
        for columna, valor in nuevos_datos.items():
            if columna in hoja['datos'].columns:
                hoja['datos'].loc[indice, columna] = valor

        hoja['metadatos']['ultima_modificacion'] = datetime.now()

        print(f" Fila {indice} actualizada en '{nombre_hoja}'")

        # Verificar triggers
        self.verificar_triggers(nombre_hoja, 'fila_actualizada', {'indice': indice, 'datos': nuevos_datos})

        return True

    def obtener_datos(self, nombre_hoja, filtros=None):
        """Obtener datos de la hoja con filtros opcionales"""
        if nombre_hoja not in self.hojas:
            print(f" Hoja '{nombre_hoja}' no existe")
            return None

        datos = self.hojas[nombre_hoja]['datos'].copy()

        if filtros:
            for columna, condicion in filtros.items():
                if columna in datos.columns:
                    if isinstance(condicion, dict):
                        operador = condicion.get('operador', '==')
                        valor = condicion.get('valor')

                        if operador == '==':
                            datos = datos[datos[columna] == valor]
                        elif operador == '>':
                            datos = datos[datos[columna] > valor]
                        elif operador == '<':
                            datos = datos[datos[columna] < valor]
                        elif operador == 'contains':
                            datos = datos[datos[columna].str.contains(str(valor), na=False)]
                    else:
                        datos = datos[datos[columna] == condicion]

        return datos

    def configurar_trigger(self, nombre_hoja, evento, callback):
        """Configurar un trigger para eventos en la hoja"""
        trigger = {
            'hoja': nombre_hoja,
            'evento': evento,  # 'nueva_fila', 'fila_actualizada', 'datos_cambiados'
            'callback': callback
        }

        self.triggers_configurados.append(trigger)
        print(f" Trigger configurado: {evento} en '{nombre_hoja}'")

    def verificar_triggers(self, nombre_hoja, evento, datos):
        """Verificar y ejecutar triggers configurados"""
        triggers_relevantes = [
            t for t in self.triggers_configurados
            if t['hoja'] == nombre_hoja and t['evento'] == evento
        ]

        for trigger in triggers_relevantes:
            try:
                trigger['callback'](nombre_hoja, evento, datos)
            except Exception as e:
                print(f" Error ejecutando trigger: {e}")

    def generar_reporte(self, nombre_hoja, tipo_reporte='resumen'):
        """Generar diferentes tipos de reportes"""
        if nombre_hoja not in self.hojas:
            print(f" Hoja '{nombre_hoja}' no existe")
            return None

        datos = self.hojas[nombre_hoja]['datos']

        if datos.empty:
            return " No hay datos para generar reporte"

        reporte = {
            'hoja': nombre_hoja,
            'fecha_generacion': datetime.now(),
            'total_filas': len(datos)
        }

        if tipo_reporte == 'resumen':
            reporte['resumen'] = {
                'columnas': list(datos.columns),
                'tipos_datos': datos.dtypes.to_dict(),
                'valores_nulos': datos.isnull().sum().to_dict(),
                'primeras_filas': datos.head().to_dict('records')
            }

        elif tipo_reporte == 'estadisticas':
            columnas_numericas = datos.select_dtypes(include=[np.number]).columns
            if len(columnas_numericas) > 0:
                reporte['estadisticas'] = datos[columnas_numericas].describe().to_dict()

        elif tipo_reporte == 'completo':
            reporte['datos_completos'] = datos.to_dict('records')

        return reporte

    def mostrar_estado(self):
        """Mostrar estado actual de todas las hojas"""
        print("\n **ESTADO DE GOOGLE SHEETS SIMULATOR**")
        print("=" * 50)

        if not self.hojas:
            print(" No hay hojas creadas")
            return

        for nombre, info in self.hojas.items():
            metadatos = info['metadatos']
            print(f"\n **{nombre}**")
            print(f"    Dimensiones: {metadatos['filas']} filas x {metadatos['columnas']} columnas")
            print(f"    Creada: {metadatos['creada'].strftime('%Y-%m-%d %H:%M')}")
            print(f"    Última modificación: {metadatos['ultima_modificacion'].strftime('%Y-%m-%d %H:%M')}")

            if not info['datos'].empty:
                print(f"    Columnas: {', '.join(info['datos'].columns)}")

# ============================================================================
# PARTE 2: SIMULADOR DE ENVÍO DE EMAILS
# ============================================================================

class EmailSimulator:
    """Simulador de envío de emails para fines educativos"""

    def __init__(self):
        self.emails_enviados = []
        self.plantillas = {}
        self.configuracion = {
            'remitente': 'sistema@empresa.com',
            'servidor_smtp': 'smtp.gmail.com',
            'puerto': 587
        }

    def crear_plantilla(self, nombre, asunto, cuerpo_html, cuerpo_texto=None):
        """Crear una plantilla de email"""
        self.plantillas[nombre] = {
            'asunto': asunto,
            'cuerpo_html': cuerpo_html,
            'cuerpo_texto': cuerpo_texto or self.html_a_texto(cuerpo_html),
            'creada': datetime.now()
        }

        print(f" Plantilla '{nombre}' creada exitosamente")

    def html_a_texto(self, html):
        """Convertir HTML básico a texto plano"""
        import re
        # Remover tags HTML básicos
        texto = re.sub('<[^<]+?>', '', html)
        return texto.strip()

    def procesar_plantilla(self, plantilla, datos):
        """Procesar plantilla con datos dinámicos"""
        asunto = plantilla['asunto']
        cuerpo_html = plantilla['cuerpo_html']
        cuerpo_texto = plantilla['cuerpo_texto']

        # Reemplazar placeholders
        for campo, valor in datos.items():
            placeholder = f"{{{{{campo}}}}}"
            asunto = asunto.replace(placeholder, str(valor))
            cuerpo_html = cuerpo_html.replace(placeholder, str(valor))
            cuerpo_texto = cuerpo_texto.replace(placeholder, str(valor))

        return {
            'asunto': asunto,
            'cuerpo_html': cuerpo_html,
            'cuerpo_texto': cuerpo_texto
        }

    def enviar_email(self, destinatario, plantilla_nombre, datos, adjuntos=None):
        """Simular envío de email"""
        if plantilla_nombre not in self.plantillas:
            print(f" Plantilla '{plantilla_nombre}' no existe")
            return False

        plantilla = self.plantillas[plantilla_nombre]
        email_procesado = self.procesar_plantilla(plantilla, datos)

        # Simular envío
        email_info = {
            'id': len(self.emails_enviados) + 1,
            'destinatario': destinatario,
            'remitente': self.configuracion['remitente'],
            'asunto': email_procesado['asunto'],
            'cuerpo_html': email_procesado['cuerpo_html'],
            'cuerpo_texto': email_procesado['cuerpo_texto'],
            'adjuntos': adjuntos or [],
            'timestamp': datetime.now(),
            'estado': 'enviado'
        }

        self.emails_enviados.append(email_info)

        print(f" **EMAIL ENVIADO**")
        print(f"    Para: {destinatario}")
        print(f"    Asunto: {email_procesado['asunto']}")
        print(f"    Adjuntos: {len(adjuntos) if adjuntos else 0}")
        print(f"    Enviado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

        return True

    def enviar_email_masivo(self, lista_destinatarios, plantilla_nombre, datos_personalizados):
        """Enviar email a múltiples destinatarios con datos personalizados"""
        emails_exitosos = 0
        emails_fallidos = 0

        print(f" **INICIANDO ENVÍO MASIVO**")
        print(f"    Destinatarios: {len(lista_destinatarios)}")
        print(f"    Plantilla: {plantilla_nombre}")

        for destinatario in lista_destinatarios:
            # Obtener datos específicos para este destinatario
            datos_dest = datos_personalizados.get(destinatario, {})

            if self.enviar_email(destinatario, plantilla_nombre, datos_dest):
                emails_exitosos += 1
            else:
                emails_fallidos += 1

        print(f"\n **ENVÍO MASIVO COMPLETADO**")
        print(f"    Exitosos: {emails_exitosos}")
        print(f"    Fallidos: {emails_fallidos}")

        return emails_exitosos, emails_fallidos

    def generar_reporte_emails(self):
        """Generar reporte de emails enviados"""
        if not self.emails_enviados:
            return " No se han enviado emails"

        df_emails = pd.DataFrame(self.emails_enviados)

        reporte = {
            'total_emails': len(self.emails_enviados),
            'emails_hoy': len(df_emails[df_emails['timestamp'].dt.date == datetime.now().date()]),
            'destinatarios_unicos': df_emails['destinatario'].nunique(),
            'plantillas_usadas': len(df_emails['asunto'].unique()),
            'ultimos_5': df_emails.tail()[['destinatario', 'asunto', 'timestamp']].to_dict('records')
        }

        return reporte

    def mostrar_estadisticas(self):
        """Mostrar estadísticas de emails"""
        print("\n **ESTADÍSTICAS DE EMAILS**")
        print("=" * 40)

        if not self.emails_enviados:
            print(" No se han enviado emails")
            return

        reporte = self.generar_reporte_emails()

        print(f" Total emails enviados: {reporte['total_emails']}")
        print(f" Emails enviados hoy: {reporte['emails_hoy']}")
        print(f" Destinatarios únicos: {reporte['destinatarios_unicos']}")
        print(f" Plantillas utilizadas: {reporte['plantillas_usadas']}")

        print(f"\n **Últimos 5 emails:**")
        for email in reporte['ultimos_5']:
            timestamp = email['timestamp'].strftime('%H:%M:%S')
            print(f"   {timestamp} - {email['destinatario']} - {email['asunto']}")

# ============================================================================
# PARTE 3: AUTOMATIZACIÓN INTEGRADA SHEETS + EMAIL
# ============================================================================

crear_banner("AUTOMATIZACIÓN INTEGRADA: SHEETS + EMAIL", "🔄")

class AutomatizacionSheetsEmail:
    """Sistema integrado de automatización con Sheets y Email"""

    def __init__(self):
        self.sheets = GoogleSheetsSimulator()
        self.email = EmailSimulator()
        self.automatizaciones_activas = []

    def configurar_reporte_automatico(self, nombre_hoja, plantilla_email, destinatarios, frecuencia='diario'):
        """Configurar reporte automático desde Google Sheets"""

        # Crear plantilla de email si no existe
        if 'reporte_ventas' not in self.email.plantillas:
            self.email.crear_plantilla(
                'reporte_ventas',
                'Reporte Diario de Ventas - {{fecha}}',
                '''
                <h2> Reporte de Ventas</h2>
                <p><strong>Fecha:</strong> {{fecha}}</p>
                <p><strong>Total de ventas:</strong> ${{total_ventas}}</p>
                <p><strong>Número de transacciones:</strong> {{num_transacciones}}</p>
                <p><strong>Producto más vendido:</strong> {{producto_top}}</p>

                <h3> Resumen por categoría:</h3>
                {{resumen_categorias}}

                <p><em>Reporte generado automáticamente el {{timestamp}}</em></p>
                '''
            )

        automatizacion = {
            'id': len(self.automatizaciones_activas) + 1,
            'nombre': f'Reporte {nombre_hoja}',
            'hoja': nombre_hoja,
            'plantilla': plantilla_email,
            'destinatarios': destinatarios,
            'frecuencia': frecuencia,
            'activa': True,
            'ultima_ejecucion': None,
            'proxima_ejecucion': self.calcular_proxima_ejecucion(frecuencia)
        }

        self.automatizaciones_activas.append(automatizacion)

        print(f" Automatización configurada: {automatizacion['nombre']}")
        print(f"    Hoja: {nombre_hoja}")
        print(f"    Destinatarios: {len(destinatarios)}")
        print(f"    Frecuencia: {frecuencia}")

        return automatizacion['id']

    def calcular_proxima_ejecucion(self, frecuencia):
        """Calcular próxima ejecución basada en frecuencia"""
        ahora = datetime.now()

        if frecuencia == 'diario':
            return ahora + timedelta(days=1)
        elif frecuencia == 'semanal':
            return ahora + timedelta(weeks=1)
        elif frecuencia == 'mensual':
            return ahora + timedelta(days=30)
        else:
            return ahora + timedelta(hours=1)

    def ejecutar_reporte_automatico(self, automatizacion_id):
        """Ejecutar un reporte automático específico"""
        automatizacion = next(
            (a for a in self.automatizaciones_activas if a['id'] == automatizacion_id),
            None
        )

        if not automatizacion:
            print(f" Automatización {automatizacion_id} no encontrada")
            return False

        print(f" **EJECUTANDO AUTOMATIZACIÓN: {automatizacion['nombre']}**")
        print("=" * 60)

        # Obtener datos de la hoja
        datos_hoja = self.sheets.obtener_datos(automatizacion['hoja'])

        if datos_hoja is None or datos_hoja.empty:
            print(f" No hay datos en la hoja '{automatizacion['hoja']}'")
            return False

        # Generar métricas del reporte
        metricas = self.calcular_metricas_reporte(datos_hoja)

        # Preparar datos para el email
        datos_email = {
            'fecha': datetime.now().strftime('%Y-%m-%d'),
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            **metricas
        }

        # Enviar email a cada destinatario
        for destinatario in automatizacion['destinatarios']:
            self.email.enviar_email(
                destinatario,
                automatizacion['plantilla'],
                datos_email
            )

        # Actualizar automatización
        automatizacion['ultima_ejecucion'] = datetime.now()
        automatizacion['proxima_ejecucion'] = self.calcular_proxima_ejecucion(automatizacion['frecuencia'])

        print(f" Reporte enviado a {len(automatizacion['destinatarios'])} destinatarios")

        return True

    def calcular_metricas_reporte(self, datos):
        """Calcular métricas para el reporte"""
        metricas = {}

        # Métricas básicas
        metricas['num_transacciones'] = len(datos)

        # Si hay columna de ventas/total
        if 'total' in datos.columns:
            metricas['total_ventas'] = f"{datos['total'].sum():,.2f}"
        elif 'precio' in datos.columns and 'cantidad' in datos.columns:
            total = (datos['precio'] * datos['cantidad']).sum()
            metricas['total_ventas'] = f"{total:,.2f}"
        else:
            metricas['total_ventas'] = "N/A"

        # Producto más vendido
        if 'producto' in datos.columns:
            if 'cantidad' in datos.columns:
                producto_top = datos.groupby('producto')['cantidad'].sum().idxmax()
            else:
                producto_top = datos['producto'].mode().iloc[0] if not datos['producto'].mode().empty else "N/A"
            metricas['producto_top'] = producto_top
        else:
            metricas['producto_top'] = "N/A"

        # Resumen por categorías
        if 'categoria' in datos.columns:
            if 'total' in datos.columns:
                resumen_cat = datos.groupby('categoria')['total'].sum()
            elif 'precio' in datos.columns and 'cantidad' in datos.columns:
                datos_temp = datos.copy()
                datos_temp['total_calc'] = datos_temp['precio'] * datos_temp['cantidad']
                resumen_cat = datos_temp.groupby('categoria')['total_calc'].sum()
            else:
                resumen_cat = datos.groupby('categoria').size()

            # Convertir a HTML
            html_categorias = "<ul>"
            for categoria, valor in resumen_cat.items():
                html_categorias += f"<li>{categoria}: ${valor:,.2f}</li>"
            html_categorias += "</ul>"
            metricas['resumen_categorias'] = html_categorias
        else:
            metricas['resumen_categorias'] = "<p>No hay datos de categorías</p>"

        return metricas

    def configurar_alerta_automatica(self, nombre_hoja, condicion, plantilla_alerta, destinatarios):
        """Configurar alerta automática basada en condiciones"""

        def callback_alerta(hoja, evento, datos):
            """Callback que se ejecuta cuando se cumple la condición"""
            print(f" **ALERTA ACTIVADA en {hoja}**")

            # Evaluar condición
            if self.evaluar_condicion_alerta(hoja, condicion):
                # Enviar alerta
                datos_alerta = {
                    'fecha': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    'hoja': hoja,
                    'evento': evento,
                    'datos_trigger': str(datos)
                }

                for destinatario in destinatarios:
                    self.email.enviar_email(destinatario, plantilla_alerta, datos_alerta)

                print(f" Alerta enviada a {len(destinatarios)} destinatarios")

        # Configurar trigger en la hoja
        self.sheets.configurar_trigger(nombre_hoja, 'nueva_fila', callback_alerta)
        self.sheets.configurar_trigger(nombre_hoja, 'fila_actualizada', callback_alerta)

        print(f" Alerta configurada para '{nombre_hoja}'")
        print(f"    Condición: {condicion}")
        print(f"    Destinatarios: {len(destinatarios)}")

    def evaluar_condicion_alerta(self, nombre_hoja, condicion):
        """Evaluar si se cumple la condición de alerta"""
        datos = self.sheets.obtener_datos(nombre_hoja)

        if datos is None or datos.empty:
            return False

        # Ejemplos de condiciones
        if condicion['tipo'] == 'valor_alto':
            columna = condicion['columna']
            umbral = condicion['umbral']
            if columna in datos.columns:
                return datos[columna].max() > umbral

        elif condicion['tipo'] == 'muchas_filas':
            umbral = condicion['umbral']
            return len(datos) > umbral

        elif condicion['tipo'] == 'valor_bajo_stock':
            columna = condicion.get('columna', 'stock')
            umbral = condicion.get('umbral', 10)
            if columna in datos.columns:
                return (datos[columna] < umbral).any()

        return False

    def mostrar_dashboard_automatizaciones(self):
        """Mostrar dashboard de todas las automatizaciones"""
        print("\n **DASHBOARD DE AUTOMATIZACIONES**")
        print("=" * 60)

        if not self.automatizaciones_activas:
            print(" No hay automatizaciones configuradas")
            return

        for auto in self.automatizaciones_activas:
            estado = " Activa" if auto['activa'] else " Inactiva"
            print(f"\n  **{auto['nombre']}** ({estado})")
            print(f"    Hoja: {auto['hoja']}")
            print(f"    Destinatarios: {len(auto['destinatarios'])}")
            print(f"    Frecuencia: {auto['frecuencia']}")

            if auto['ultima_ejecucion']:
                print(f"    Última ejecución: {auto['ultima_ejecucion'].strftime('%Y-%m-%d %H:%M')}")

            print(f"   ⏭ Próxima ejecución: {auto['proxima_ejecucion'].strftime('%Y-%m-%d %H:%M')}")

# ============================================================================
# PARTE 4: DEMOSTRACIÓN COMPLETA
# ============================================================================

print(" **DEMOSTRACIÓN COMPLETA: SHEETS + EMAIL**")
print("=" * 60)

# Crear sistema integrado
sistema = AutomatizacionSheetsEmail()

# Paso 1: Crear hoja de ventas con datos de ejemplo
print("\n 1 **CREANDO HOJA DE VENTAS CON DATOS**")
datos_ventas = pd.DataFrame({
    'fecha': pd.date_range('2024-01-01', periods=20, freq='D'),
    'producto': np.random.choice(['Laptop', 'Mouse', 'Teclado', 'Monitor', 'Tablet'], 20),
    'categoria': np.random.choice(['Computadoras', 'Accesorios'], 20),
    'precio': np.random.uniform(25, 1500, 20).round(2),
    'cantidad': np.random.randint(1, 5, 20),
    'vendedor': np.random.choice(['Ana García', 'Luis Martín', 'María López'], 20)
})

# Calcular total
datos_ventas['total'] = datos_ventas['precio'] * datos_ventas['cantidad']

sistema.sheets.crear_hoja('Ventas_Enero', datos_ventas)

print("\n **Primeras 5 filas de datos:**")
print(datos_ventas.head())

# Paso 2: Configurar plantilla de alerta
print("\n 2 **CONFIGURANDO PLANTILLA DE ALERTA**")
sistema.email.crear_plantilla(
    'alerta_venta_alta',
    ' ALERTA: Venta de alto valor detectada',
    '''
    <h2> Alerta de Venta de Alto Valor</h2>
    <p><strong>Fecha y hora:</strong> {{fecha}}</p>
    <p><strong>Hoja afectada:</strong> {{hoja}}</p>
    <p><strong>Evento:</strong> {{evento}}</p>

    <p>Se ha detectado una transacción que requiere atención.</p>

    <p><strong>Detalles del trigger:</strong></p>
    <pre>{{datos_trigger}}</pre>

    <p><em>Esta es una alerta automática del sistema.</em></p>
    '''
)

# Paso 3: Configurar reporte automático
print("\n 3 **CONFIGURANDO REPORTE AUTOMÁTICO**")
destinatarios_reporte = ['gerente@empresa.com', 'ventas@empresa.com']

auto_id = sistema.configurar_reporte_automatico(
    'Ventas_Enero',
    'reporte_ventas',
    destinatarios_reporte,
    'diario'
)

# Paso 4: Configurar alerta automática
print("\n 4 **CONFIGURANDO ALERTA AUTOMÁTICA**")
condicion_alerta = {
    'tipo': 'valor_alto',
    'columna': 'total',
    'umbral': 2000
}

destinatarios_alerta = ['supervisor@empresa.com', 'seguridad@empresa.com']

sistema.configurar_alerta_automatica(
    'Ventas_Enero',
    condicion_alerta,
    'alerta_venta_alta',
    destinatarios_alerta
)

# Paso 5: Simular eventos que activan automatizaciones
print("\n 5 **SIMULANDO EVENTOS QUE ACTIVAN AUTOMATIZACIONES**")

# Agregar venta normal
print("\n Agregando venta normal...")
venta_normal = {
    'fecha': datetime.now().date(),
    'producto': 'Mouse',
    'categoria': 'Accesorios',
    'precio': 29.99,
    'cantidad': 2,
    'vendedor': 'Ana García',
    'total': 59.98
}

sistema.sheets.agregar_fila('Ventas_Enero', venta_normal)

# Agregar venta de alto valor (activará alerta)
print("\n Agregando venta de ALTO VALOR...")
venta_alta = {
    'fecha': datetime.now().date(),
    'producto': 'Servidor Empresarial',
    'categoria': 'Computadoras',
    'precio': 2500.00,
    'cantidad': 1,
    'vendedor': 'Luis Martín',
    'total': 2500.00
}

sistema.sheets.agregar_fila('Ventas_Enero', venta_alta)

# Paso 6: Ejecutar reporte manual
print("\n 6 **EJECUTANDO REPORTE MANUAL**")
sistema.ejecutar_reporte_automatico(auto_id)

# Paso 7: Mostrar dashboard
print("\n 7 **DASHBOARD FINAL**")
sistema.mostrar_dashboard_automatizaciones()

# Mostrar estadísticas de emails
sistema.email.mostrar_estadisticas()

# Mostrar estado de sheets
sistema.sheets.mostrar_estado()

# ============================================================================
# PARTE 5: CASOS DE USO ESPECÍFICOS
# ============================================================================

crear_banner("CASOS DE USO ESPECÍFICOS", "🎯")

casos_uso_sheets_email = [
    {
        "nombre": " Reporte de Ventas Diario",
        "descripcion": "Envío automático de métricas de ventas cada mañana",
        "trigger": "Programado diariamente a las 8:00 AM",
        "datos_sheets": "Ventas del día anterior",
        "email_contenido": "Resumen ejecutivo con gráficos",
        "destinatarios": "Gerencia y equipo comercial",
        "beneficio": "Decisiones informadas desde temprano"
    },
    {
        "nombre": " Alerta de Inventario Bajo",
        "descripcion": "Notificación cuando productos tienen stock crítico",
        "trigger": "Stock < 10 unidades",
        "datos_sheets": "Inventario actualizado en tiempo real",
        "email_contenido": "Lista de productos a reabastecer",
        "destinatarios": "Compras y almacén",
        "beneficio": "Evita quiebres de stock"
    },
    {
        "nombre": " Seguimiento de Leads",
        "descripcion": "Notificación de nuevos leads y seguimiento",
        "trigger": "Nuevo lead en formulario web",
        "datos_sheets": "Base de datos de prospectos",
        "email_contenido": "Información del lead y próximos pasos",
        "destinatarios": "Equipo de ventas",
        "beneficio": "Respuesta rápida a oportunidades"
    },
    {
        "nombre": " Reporte Financiero Semanal",
        "descripcion": "Resumen financiero cada viernes",
        "trigger": "Programado semanalmente",
        "datos_sheets": "Ingresos, gastos, márgenes",
        "email_contenido": "Dashboard financiero completo",
        "destinatarios": "CFO y contabilidad",
        "beneficio": "Control financiero constante"
    },
    {
        "nombre": " Onboarding de Empleados",
        "descripcion": "Proceso automático para nuevos empleados",
        "trigger": "Nuevo empleado en RRHH sheet",
        "datos_sheets": "Datos del empleado y departamento",
        "email_contenido": "Bienvenida y checklist de tareas",
        "destinatarios": "Empleado, supervisor, RRHH",
        "beneficio": "Onboarding consistente y completo"
    }
]

for i, caso in enumerate(casos_uso_sheets_email, 1):
    print(f"{i}. **{caso['nombre']}**")
    print(f"    {caso['descripcion']}")
    print(f"    Trigger: {caso['trigger']}")
    print(f"    Datos: {caso['datos_sheets']}")
    print(f"    Email: {caso['email_contenido']}")
    print(f"    Para: {caso['destinatarios']}")
    print(f"    Beneficio: {caso['beneficio']}")
    print()

# ============================================================================
# PARTE 6: MEJORES PRÁCTICAS
# ============================================================================

crear_banner("MEJORES PRÁCTICAS SHEETS + EMAIL", "⭐")

print(" **MEJORES PRÁCTICAS PARA AUTOMATIZACIÓN SHEETS + EMAIL**")
print()

mejores_practicas_sheets_email = {
    " DISEÑO DE HOJAS": [
        "Usa nombres de columnas claros y consistentes",
        "Incluye columnas de timestamp para auditoría",
        "Mantén datos limpios y validados",
        "Usa formatos consistentes (fechas, números)",
        "Crea hojas separadas para diferentes propósitos"
    ],
    " PLANTILLAS DE EMAIL": [
        "Diseña plantillas responsive (móvil-friendly)",
        "Usa asuntos descriptivos y dinámicos",
        "Incluye tanto versión HTML como texto plano",
        "Personaliza con datos específicos del destinatario",
        "Mantén el contenido conciso y accionable"
    ],
    " AUTOMATIZACIÓN": [
        "Configura filtros para evitar spam",
        "Usa frecuencias apropiadas (no sobre-notificar)",
        "Implementa manejo de errores robusto",
        "Mantén logs de todas las ejecuciones",
        "Prueba con datos reales antes de activar"
    ],
    " SEGURIDAD": [
        "Limita acceso a hojas sensibles",
        "Usa listas de destinatarios controladas",
        "No incluyas información confidencial en emails",
        "Implementa autenticación en APIs",
        "Audita regularmente los accesos"
    ],
    " MONITOREO": [
        "Revisa métricas de entrega de emails",
        "Monitorea el rendimiento de las hojas",
        "Configura alertas para fallos del sistema",
        "Mantén estadísticas de uso",
        "Solicita feedback de los usuarios"
    ]
}

for categoria, practicas in mejores_practicas_sheets_email.items():
    print(f"{categoria}")
    for practica in practicas:
        print(f"    {practica}")
    print()

print("\n" + "="*80)
print(" **RESUMEN: GOOGLE SHEETS + EMAIL AUTOMATION**")
print("="*80)
print()
print(" **Lo que hemos aprendido:**")
print("    Simulación de Google Sheets como base de datos")
print("    Sistema de envío de emails con plantillas")
print("    Automatización integrada Sheets + Email")
print("    Configuración de alertas automáticas")
print("    Reportes automáticos programados")
print("    Casos de uso específicos por industria")
print("    Mejores prácticas para implementación")
print()
print(" **Componentes desarrollados:**")
print("    GoogleSheetsSimulator - Simulador de Google Sheets")
print("    EmailSimulator - Sistema de envío de emails")
print("    AutomatizacionSheetsEmail - Sistema integrado")
print("    Dashboard de monitoreo")
print("    Sistema de alertas automáticas")
print()
print(" **Próximo paso:**")
print("   En el siguiente notebook aprenderemos sobre")
print("   herramientas adicionales como Slack y programación")
print("   de tareas recurrentes más avanzadas")
print()
print(" **Para implementación real:**")
print("   1. Configura Google Sheets API")
print("   2. Configura SMTP para envío de emails")
print("   3. Usa Zapier/Make para conectar sin código")
print("   4. Implementa gradualmente, empezando simple")
print("   5. Monitorea y optimiza constantemente")


 📊 GOOGLE SHEETS + EMAIL: AUTOMATIZACIÓN PODEROSA 📊
 **¿Por qué Google Sheets + Email?**
   • Google Sheets: Base de datos accesible y colaborativa
   • Email: Canal de comunicación universal
   • Combinación: Reportes automáticos y actualizaciones en tiempo real

 **GOOGLE SHEETS COMO CENTRO DE DATOS**

 🔄 AUTOMATIZACIÓN INTEGRADA: SHEETS + EMAIL 🔄
 **DEMOSTRACIÓN COMPLETA: SHEETS + EMAIL**

 1 **CREANDO HOJA DE VENTAS CON DATOS**
 Hoja 'Ventas_Enero' creada exitosamente
    Dimensiones: 20 filas x 7 columnas

 **Primeras 5 filas de datos:**
       fecha producto     categoria  precio  cantidad     vendedor    total
0 2024-01-01   Tablet  Computadoras   35.01         1   Ana García    35.01
1 2024-01-02   Laptop  Computadoras  417.05         3   Ana García  1251.15
2 2024-01-03    Mouse    Accesorios  526.49         2  Luis Martín  1052.98
3 2024-01-04  Teclado  Computadoras  360.84         2   Ana García   721.68
4 2024-01-05   Laptop  Computadoras  642.35         3   Ana García  19